In [5]:
import torch
from torch_dataset import ZootrDataset 
import os
from pytorch_efficientnet import to_tensor, EfficientNetwork, eval_model, augment, augmented_pred
from tqdm import tqdm
import gc

In [6]:
mdl = torch.load('model.pth', weights_only=False)

In [8]:
path_to_frames = r"C:\Users\guilh\Git\ZOOTR\Testes1"

In [9]:
test_imgs = [os.path.join(path_to_frames, i) for i in os.listdir(path_to_frames)]

In [10]:
test_set = ZootrDataset(test_imgs, False, augmented_pred)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=2, num_workers=0, drop_last=False, shuffle=False)

In [12]:
all_preds = torch.empty(0, 2)
for imgs in tqdm(test_loader):
    with torch.no_grad():
        imgs_pred = mdl(imgs)
        all_preds = torch.cat((all_preds, imgs_pred))
    gc.collect()

100%|██████████| 23440/23440 [1:26:25<00:00,  4.52it/s]


In [13]:
preds = torch.nn.Softmax()(all_preds)

c:\Users\guilh\Git\ZOOTR\.venv_zootr\Lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [25]:
import numpy as np

inds = np.where(preds[:,1] > 0.75)[0]
breaks = np.where(np.diff(inds) > 30)[0]

In [26]:
breaks = np.where(np.diff(inds) > 30)[0]

In [16]:
items_inds = inds[breaks]

In [20]:
len(items_inds)

18

In [24]:
from skimage import io
from torchvision import transforms

for i in range(len(items_inds)):
    image = io.imread(test_imgs[items_inds[i]])
    transforms.ToPILImage()(image).show()

In [ ]:
import torch
import torch.optim as optim
from torch_dataset import ZootrDataset 
import numpy as np
import os
from pytorch_efficientnet import to_tensor, EfficientNetwork, eval_model, augment, augmented_pred
import random

random.seed(8)
load_data_objects = False
save_data_objects = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

item_path = r"C:\Users\guilh\Git\ZOOTR\Test\Items\\"
non_item_path = r"C:\Users\guilh\Git\ZOOTR\Random_frames\\"


if  load_data_objects:
    torch.load('train_data.pth')
    torch.load('valid_data.pth')
else:
    items_images = [item_path + i for i in os.listdir(item_path)]
    #Get all items
    non_items_images = [non_item_path + i for i in os.listdir(non_item_path)]
    #Get non items
    all_images = list(np.random.permutation(items_images + non_items_images))
    #Combine all images' paths and shuffle
    train_data = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augment)
    train_data_eval = ZootrDataset(all_images[0:int(np.ceil(len(all_images)*0.8))], transform_=augmented_pred)
    valid_data = ZootrDataset(all_images[int(np.ceil(len(all_images)*0.8)):], transform_=augmented_pred)
    # Data Objects
    if save_data_objects:
        torch.save(train_data, 'train_data.pth')
        torch.save(valid_data, 'valid_data.pth')
    
    
batch_size = 20
num_workers = 0

data = {'train': all_images[0:int(np.ceil(len(all_images)*0.7))], 'val': all_images[int(np.ceil(len(all_images)*0.7)):int(np.ceil(len(all_images)*0.8))], 
        'test': all_images[int(np.ceil(len(all_images)*0.8)):]}
transformations = {'train': augment, 'val': augmented_pred, 'test': augmented_pred}
dataset_sizes = {'train': int(np.ceil(len(all_images)*0.7)), 'val': int(np.ceil(len(all_images)*0.8)) - int(np.ceil(len(all_images)*0.7)), 
                 'test': len(all_images) - int(np.ceil(len(all_images)*0.8))}

datasets = {x: ZootrDataset(data[x], transform_=transformations[x]) for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(datasets[x], batch_size=batch_size, num_workers=num_workers, drop_last=True, shuffle=False) for x in ['train', 'val', 'test']}



In [ ]:
io.imread(items_images[0]).shape

(1080, 1920, 3)

In [ ]:
import os
import numpy as np
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ZootrDataset(Dataset):

    def __init__(self, instance_list, is_train=True, transform_=None):        
        self.is_train = is_train
        if self.is_train:
            self.instance_list = np.random.permutation(instance_list)
        else:
            self.instance_list = instance_list
        self.transform_ = transform_

    def __len__(self):
        return len(self.instance_list)

    def __getitem__(self, idx):
        img_name = self.instance_list[idx]
        image = io.imread(img_name)

        if self.transform_:
            image = self.transform_(image)


        if self.is_train:
            label = int('Random' not in self.instance_list[idx])
            return image, label
        else:
            return image

    
    def see_image(self, name):
        if type(name) == int:
            im, _ = self.__getitem__(name)
            transforms.ToPILImage()(im)
        else:
            image = io.imread(name)
            io.imshow(image)
       

In [1]:
from functions import get_frames_from_video

In [3]:
import cv2
from functions import shave_black_bit, get_frames_from_video
from PIL import Image
from torch.autograd import Variable
from torchvision import transforms
from pytorch_efficientnet import to_tensor, EfficientNetwork, eval_model, augment, augmented_pred
import torch
from aux_funs import save_video_frames


vid = r'C:\Users\guilh\Videos\Captures\THE LEGEND OF ZELDA - Project64 3.0.1.5664-2df3434 2025-07-20 19-53-09.mp4'

predictions = save_video_frames(vid, 'Testes2', 1, None, to_tensor)

In [4]:
predictions

[]